<a href="https://colab.research.google.com/github/JiaweiYu1/fasttext/blob/master/yelp_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import re
import os
import codecs

import tensorflow as tf

import time
import datetime

from sklearn.utils import shuffle
import csv
import html

from tensorflow.contrib import slim

import argparse
from datetime import datetime
import pickle
import pprint
import easydict
import random


#tf.enable_eager_execution()

In [13]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/My Drive/Colab_Notebooks/fasttext")
cwd = os.getcwd()
print(cwd)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab_Notebooks/fasttext


In [0]:
class fastTextModel():
    """
    A simple implementation of fasttext for text classification
    """
    def __init__(self, sequence_length, num_classes, vocab_size,
                 embedding_size, learning_rate, l2_reg_lambda, epoch, 
                 decay_steps, decay_rate, is_training=True,
                 initializer=tf.random_normal_initializer(stddev=0.1)):
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.num_classes = num_classes
        self.sequence_length = sequence_length
        self.learning_rate = learning_rate
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate
        self.epoch = epoch
        self.is_training = is_training
        self.l2_reg_lambda = l2_reg_lambda
        self.initializer = initializer
 
        self.input_x = tf.placeholder(tf.int32, [None, self.sequence_length], name='input_x')
        self.input_y = tf.placeholder(tf.int32, [None, self.num_classes], name='input_y')
 
        self.global_step = tf.Variable(0, trainable=False, name='global_step')
        self.instantiate_weight()
        self.logits = self.inference()
        self.loss_val = self.loss()
        self.train_op = self.train()
 
        self.predictions = tf.argmax(self.logits, axis=1, name='predictions')
        correct_prediction = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'), name='accuracy')
 
    def instantiate_weight(self):
        with tf.variable_scope('weights', reuse=tf.AUTO_REUSE):
            self.Embedding = tf.get_variable('Embedding', shape=[self.vocab_size, self.embedding_size],
                                             initializer=self.initializer)
            self.W_projection = tf.get_variable('W_projection', shape=[self.embedding_size, self.num_classes],
                                                initializer=self.initializer)
            self.b_projection = tf.get_variable('b_projection', shape=[self.num_classes])
 
 
    def inference(self):
        with tf.name_scope('embedding'):
            words_embedding = tf.nn.embedding_lookup(self.Embedding, self.input_x)
            self.average_embedding = tf.reduce_mean(words_embedding, axis=1)
 
        logits = tf.matmul(self.average_embedding, self.W_projection) +self.b_projection
 
        return logits
 
 
    def loss(self):
        # loss
        with tf.name_scope('loss'):
            losses = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.input_y, logits=self.logits)
            data_loss = tf.reduce_mean(losses)
            #l2_loss = tf.add_n([tf.nn.l2_loss(cand_var) for cand_var in tf.trainable_variables()
            #                    if 'bias' not in cand_var.name]) * self.l2_reg_lambda
            #data_loss += l2_loss * self.l2_reg_lambda
            return data_loss
 
    def train(self):
        with tf.name_scope('train'):
            learning_rate = tf.train.exponential_decay(self.learning_rate, self.global_step,
                                                       self.decay_steps, self.decay_rate,
                                                       staircase=True)
 
            train_op = tf.contrib.layers.optimize_loss(self.loss_val, global_step=self.global_step,
                                                      learning_rate=learning_rate, optimizer='Adam')
 
        return train_op

In [4]:

class_size=2
learning_rate=0.1
batch_size=32
decay_steps=1000
decay_rate=0.9
sequence_length=467
vocab_size = 4939525
embedding_dim = 10
is_training=True
dropout_keep_prob=1
epoch = 5
batch_size = 4096
l2_reg_lambda = 0.01


fasttext = fastTextModel(sequence_length,
                      class_size,
                      vocab_size,
                      embedding_dim,
                      learning_rate,
                      l2_reg_lambda,
                      epoch,
                      decay_steps,
                      decay_rate,
                      is_training=True,
                    )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
def batch_iter(x, y, batch_size=1, shuffle=True):
  data_x = np.array(x)
  data_y = np.array(y)
  data_len = len(x)
  num_batch = int((data_len-1)/batch_size)+1
    
  if shuffle:
    shuffle_indices = np.random.permutation(np.arange(data_len))
    x_shuffle = data_x[shuffle_indices]
    y_shuffle= data_y[shuffle_indices]
    
  else:
    x_shuffle=x
    y_shuffle = y
  for i in range(num_batch):
    start_index = i*batch_size
    end_index = min((i+1)*batch_size, data_len)
    yield (x_shuffle[start_index:end_index], y_shuffle[start_index:end_index])
    
    
    
def fit(train_x, train_y, x_dev, y_dev, batch_size, epoch):
  if not os.path.exists('yelp_review_polarity_csv/saves/fasttext'): os.makedirs('yelp_review_polarity_csv/saves/fasttext')
  if not os.path.exists('yelp_review_polarity_csv/train_logs/fasttext'): os.makedirs('yelp_review_polarity_csv/train_logs/fasttext')

  train_steps = 0
  best_val_acc = 0
    
  tf.summary.scalar('val_loss', fasttext.loss_val)
  tf.summary.scalar('val_accuracy', fasttext.accuracy)
  merged = tf.summary.merge_all()
    
  sess = tf.Session()
    
  writer = tf.summary.FileWriter('yelp_review_polarity_csv/train_logs/fasttext', sess.graph)
  saver = tf.train.Saver(max_to_keep=10)
  sess.run(tf.global_variables_initializer())
    
  for i in range(epoch):
    start_time = time.time()
    batch_train = batch_iter(train_x, train_y, batch_size)
    for batch_x, batch_y in batch_train:
      train_steps +=1
      feed_dict = {fasttext.input_x:batch_x, fasttext.input_y:batch_y}
      _, train_loss, train_acc = sess.run([fasttext.train_op, fasttext.loss_val, fasttext.accuracy], feed_dict = feed_dict)
        
      if train_steps % 100 ==0:
        feed_dict = {fasttext.input_x:x_dev, fasttext.input_y:y_dev}
        val_loss, val_acc = sess.run([fasttext.loss_val, fasttext.accuracy],feed_dict=feed_dict)
        
        summary = sess.run(merged, feed_dict=feed_dict)
        writer.add_summary(summary, global_step=train_steps)
        
        if val_acc >= best_val_acc:
          best_val_acc = val_acc
          saver.save(sess, "yelp_review_polarity_csv/saves/fasttext/model.ckpt", global_step=train_steps)
        msg = 'epoch:%d/%d, train_steps:%d, train_loss:%.4f, trina_acc:%.4f, val_loss:%.4f, val_acc:%.4f'
        print(msg%(i, epoch, train_steps, train_loss, train_acc, val_loss, val_acc))
    print("--- %s seconds per epoch ---" % (time.time() - start_time))   

      
def predict(x):
  sess = tf.Session()
    
  sess.run(tf.global_variables_initializer())
  saver = tf.train.Saver(tf.global_variables())
  ckpt = tf.train.get_checkpoint_state('yelp_review_polarity_csv/saves/fasttext/')
  saver.restore(sess, ckpt.model_checkpoint_path)
  #saver.restore(sess, 'saves/fasttext/checkpoint')
    
  feed_dict={fasttext.input_x:x}
  logits = sess.run(fasttext.logits, feed_dict=feed_dict)
  y_pred = np.argmax(logits, 1)
    
  return y_pred

In [0]:
file_to_read = open('data/yelp_train_x_1.pkl', 'rb')

yelp_train_x = pickle.load(file_to_read)

file_to_read.close()

In [0]:
file_to_read = open('data/yelp_y_train_1.pkl', 'rb')

yelp_y_train = pickle.load(file_to_read)

file_to_read.close()

In [0]:
file_to_read = open('data/yelp_test_x.pkl', 'rb')

yelp_x_test = pickle.load(file_to_read)

file_to_read.close()

In [0]:
file_to_read = open('data/yelp_test_y.pkl', 'rb')

yelp_y_test = pickle.load(file_to_read)

file_to_read.close()

In [0]:
dev_sample_index = -1 * int(0.1 * float(len(yelp_y_train)))
x_train_yelp, x_dev_yelp = yelp_train_x[:dev_sample_index], yelp_train_x[dev_sample_index:]
yelp_y_train_1, yelp_y_dev_1 = yelp_y_train[:dev_sample_index], yelp_y_train[dev_sample_index:]

In [0]:
yelp_y_dev_2 = np.array(yelp_y_dev_1).astype(np.int32).tolist()
yelp_y_train_2 = np.array(yelp_y_train_1).astype(np.int32).tolist()

In [0]:
del yelp_train_x
del yelp_y_train

In [11]:
fit(x_train_yelp, yelp_y_train_2, x_dev_yelp, yelp_y_dev_2,batch_size, epoch)

epoch:0/5, train_steps:100, train_loss:0.1333, trina_acc:0.9490, val_loss:0.1347, val_acc:0.9523
--- 102.65831303596497 seconds per epoch ---
epoch:1/5, train_steps:200, train_loss:0.0450, trina_acc:0.9841, val_loss:0.1430, val_acc:0.9504
--- 96.42351388931274 seconds per epoch ---
epoch:2/5, train_steps:300, train_loss:0.0137, trina_acc:0.9973, val_loss:0.1503, val_acc:0.9532
--- 104.4518985748291 seconds per epoch ---
epoch:3/5, train_steps:400, train_loss:0.0082, trina_acc:0.9980, val_loss:0.1704, val_acc:0.9516
--- 97.58151984214783 seconds per epoch ---
epoch:4/5, train_steps:500, train_loss:0.0031, trina_acc:1.0000, val_loss:0.1835, val_acc:0.9520
epoch:4/5, train_steps:600, train_loss:0.0043, trina_acc:0.9990, val_loss:0.1967, val_acc:0.9510
--- 102.35480213165283 seconds per epoch ---


In [18]:
y_predict = predict(yelp_x_test)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from yelp_review_polarity_csv/saves/fasttext/model.ckpt-300


In [0]:
c_prediction = tf.equal(y_predict, yelp_y_test)

In [20]:
from sklearn.metrics import confusion_matrix
confusion_matrix(yelp_y_test, y_predict)

array([[18141,   859],
       [  942, 18058]])

In [21]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(yelp_y_test, y_predict)
print('The test accuracy is', accuracy)

The test accuracy is 0.9526052631578947
